<a href="https://colab.research.google.com/github/remotesensinginfo/pb_gee_tools/blob/main/examples/applications/burnt_vegetation/02_Vegetation_Recovery_Post_Fire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# Load the matplotlib library for making plots
import matplotlib.pyplot as plt

# Import the numpy module
import numpy

# Pandas allows us to create a spreadsheet output
# datasets
import pandas

# Import the scipy statistics module.
import scipy.stats

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/burnt_veg/burnt_area_roi.geojson"

# The date of the fire event
fire_date = datetime.datetime(year=2015, month=8, day=20)

# Dates before the fire
pre_burn_start_date = datetime.datetime(year=2007, month=1, day=1)
pre_burn_end_date = datetime.datetime(year=2015, month=8, day=20)

# Dates after the fire
post_burn_start_date = datetime.datetime(year=2015, month=8, day=20)
post_burn_end_date = datetime.datetime(year=2024, month=12, day=31)

# No Data Value
out_no_data_val = 0.0

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
# Get the landsat image collection
pre_burn_ls_img_col = pb_gee_tools.datasets.get_landsat_sr_collection(
    aoi=tile_aoi,
    start_date=pre_burn_start_date,
    end_date=pre_burn_end_date,
    cloud_thres=70,
    ignore_ls7=False,
    out_lstm_bands=False,
)

# Filter the collection to a specific row/path
pre_burn_ls_img_col = pre_burn_ls_img_col.filter(ee.Filter.eq("WRS_PATH", 203)).filter(
    ee.Filter.eq("WRS_ROW", 32)
)

In [ ]:
# Get the landsat image collection
post_burn_ls_img_col = pb_gee_tools.datasets.get_landsat_sr_collection(
    aoi=tile_aoi,
    start_date=post_burn_start_date,
    end_date=post_burn_end_date,
    cloud_thres=70,
    ignore_ls7=False,
    out_lstm_bands=False,
)

# Filter the collection to a specific row/path
post_burn_ls_img_col = post_burn_ls_img_col.filter(
    ee.Filter.eq("WRS_PATH", 203)
).filter(ee.Filter.eq("WRS_ROW", 32))

In [ ]:
# Specify the path to the asset exported from the first notebook
asset_id = f'projects/{ee_prj_name}/assets/burnt_veg_example/burnt_area_severity'


In [ ]:
# Load the burnt area image
burn_area_severity_img = ee.Image.load(asset_id)
# Select the first band 'burnt_area'
burnt_area_img = burn_area_severity_img.select("burnt_area")

In [ ]:
# Convert the burnt area to a vector
burnt_area_vec = burnt_area_img.reduceToVectors(
    geometry=tile_aoi,
    scale=30,
    geometryType="polygon",
    eightConnected=False,
    labelProperty="zone",
)

In [ ]:
# A function to calculate the NDVI for the Landsat images
# This function will be mapped to the image collection
# so the NDVI will be calculated for each of the Landsat
# images in a image collection.
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(["NIR", "Red"]).rename("NDVI")
    return image.addBands(ndvi)

In [ ]:
# Calculate the NDVI for each of the Landsat images pre-fire
pre_burn_ndvi_ls_img_col = pre_burn_ls_img_col.map(calculate_ndvi)
# Calculate the NDVI for each of the Landsat images post-fire
post_burn_ndvi_ls_img_col = post_burn_ls_img_col.map(calculate_ndvi)

In [ ]:
# Monitor vegetation recovery
# Create a time series of NDVI values
def create_time_series_ndvi(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=burnt_area_vec, scale=30, maxPixels=1e9
    )
    return ee.Feature(None, {"date": image.date().format(), "NDVI": stats.get("NDVI")})

In [ ]:
# Perform the zonal stats for the burnt area extracting the NDVI values
# before the fire mapping the image collection of Landsat images.
pre_burn_ndvi_time_series = pre_burn_ndvi_ls_img_col.map(
    create_time_series_ndvi
).getInfo()

# Perform the zonal stats for the burnt area extracting the NDVI values
# after the fire mapping the image collection of Landsat images.
post_burn_ndvi_time_series = post_burn_ndvi_ls_img_col.map(
    create_time_series_ndvi
).getInfo()

In [ ]:
# Convert time series to a pandas DataFrame
pre_burn_feats = pre_burn_ndvi_time_series["features"]

# Create a list of the dates and NDVI data
pre_burn_data = list()
for pre_burn_feat in pre_burn_feats:
    if "NDVI" in pre_burn_feat["properties"]:
        pre_burn_data.append(
            {
                "date": pre_burn_feat["properties"]["date"],
                "NDVI": pre_burn_feat["properties"]["NDVI"],
            }
        )

# Convert the list of a Pandas Dataframe
pre_burn_ndvi_df = pandas.DataFrame(pre_burn_data)
# Convert the date column to a datatime object
pre_burn_ndvi_df["date"] = pandas.to_datetime(pre_burn_ndvi_df["date"])
# Sort the Dataframe by the date
pre_burn_ndvi_df = pre_burn_ndvi_df.sort_values("date")
pre_burn_ndvi_df

In [ ]:
# Convert time series to a pandas DataFrame
post_burn_feats = post_burn_ndvi_time_series["features"]

# Create a list of the dates and NDVI data
post_burn_data = list()
for post_burn_feat in post_burn_feats:
    if "NDVI" in post_burn_feat["properties"]:
        post_burn_data.append(
            {
                "date": post_burn_feat["properties"]["date"],
                "NDVI": post_burn_feat["properties"]["NDVI"],
            }
        )

# Convert the list of a Pandas Dataframe
post_burn_ndvi_df = pandas.DataFrame(post_burn_data)
# Convert the date column to a datatime object
post_burn_ndvi_df["date"] = pandas.to_datetime(post_burn_ndvi_df["date"])
# Sort the Dataframe by the date
post_burn_ndvi_df = post_burn_ndvi_df.sort_values("date")
post_burn_ndvi_df

In [ ]:
# Create a plot to visualise the NDVI before and after the fire

plt.figure(figsize=(10, 6))
plt.plot(
    pre_burn_ndvi_df["date"],
    pre_burn_ndvi_df["NDVI"],
    label="Pre-fire NDVI",
    color="Green",
)
plt.axhline(
    y=pre_burn_ndvi_df["NDVI"].mean(),
    color="orange",
    linestyle="--",
    label="Pre-fire NDVI (mean)",
)

plt.plot(
    post_burn_ndvi_df["date"],
    post_burn_ndvi_df["NDVI"],
    label="Post-fire NDVI",
    color="blue",
)
plt.axhline(
    y=post_burn_ndvi_df["NDVI"].mean(),
    color="red",
    linestyle="--",
    label="Post-fire NDVI (mean)",
)

plt.title("Vegetation Recovery Following Fire")
plt.xlabel("Date")
plt.ylabel("NDVI")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Save the pandas dataframes to the google drive
pre_burn_ndvi_df.to_excel(
    "/content/drive/MyDrive/burnt_veg/pre_burn_ndvi.xlsx", index=False
)
post_burn_ndvi_df.to_excel(
    "/content/drive/MyDrive/burnt_veg/post_burn_ndvi.xlsx", index=False
)

In [ ]:
# Monitor vegetation recovery
# While the NDVI providing information on the vegetation cover
# it does not relate to the structure of the vegetation.
# L-Band SAR data relates to the vertical structure of the vegetation.
def create_time_series_palsar(image):
    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=burnt_area_vec, scale=25, maxPixels=1e9
    )
    return ee.Feature(
        None,
        {
            "date": image.date().format(),
            "HH": stats.get("HH"),
            "HV": stats.get("HV"),
        },
    )

In [ ]:
# Load the L-Band JAXA PALSAR and PALSAR-2 image collection.
palsar_img_col = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH")

In [ ]:
# Perform the zonal stats to extract the L-Band backscatter values
# for the burnt area.
palsar_time_series = palsar_img_col.map(create_time_series_palsar).getInfo()

In [ ]:
# Not needed for this notebook but for reference this is the code to convert
# the PALSAR images to dBs.
def convert_palsar_db(img):
    hh_db = img.select("HH").pow(2).log10().multiply(10).subtract(83).rename("HH_dB")
    hv_db = img.select("HV").pow(2).log10().multiply(10).subtract(83).rename("HV_dB")
    return img.addBands([hh_db, hv_db])


palsar_dB_img_col = palsar_img_col.map(convert_palsar_db).filterBounds(tile_aoi)

In [ ]:
# Convert time series to a pandas DataFrame - the same process as
# for the NDVI.
palsar_burn_feats = palsar_time_series["features"]
palsar_burn_data = list()
for palsar_burn_feat in palsar_burn_feats:
    if "HV" in palsar_burn_feat["properties"]:
        palsar_burn_data.append(
            {
                "date": palsar_burn_feat["properties"]["date"],
                "HH": palsar_burn_feat["properties"]["HH"],
                "HV": palsar_burn_feat["properties"]["HV"],
            }
        )

palsar_df = pandas.DataFrame(palsar_burn_data)
palsar_df["date"] = pandas.to_datetime(palsar_df["date"])
palsar_df = palsar_df.sort_values("date")
palsar_df

In [ ]:
# The SAR Backscatter (Power) values need converting to decibels (dB)
# This should be done after taking the mean (i.e., the zonal stats step)
# as the mean of values which have been through a log operation is not
# the same as the log of mean of the values.

palsar_df["HH_dB"] = (numpy.log10(palsar_df["HH"].pow(2)) * 10) - 83
palsar_df["HV_dB"] = (numpy.log10(palsar_df["HV"].pow(2)) * 10) - 83

In [ ]:
# Plot the SAR Backscatter for the period.

plt.figure(figsize=(10, 6))
plt.plot(palsar_df["date"], palsar_df["HH_dB"], label="HH dB", color="Green")
plt.plot(palsar_df["date"], palsar_df["HV_dB"], label="HV dB", color="blue")

plt.axvline(x=fire_date, color="red", linestyle="--", label="Fire")

plt.title("Vegetation Recovery Following Fire")
plt.xlabel("Date")
plt.ylabel("dB")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Save the backscatter data to your google drive.
palsar_df.to_excel(
    "/content/drive/MyDrive/burnt_veg/palsar_timeseries.xlsx", index=False
)

In [ ]:
# Create a plot with both the NDVI and Backscatter:

fig, ax1 = plt.subplots(figsize=(12, 8))
ax1.plot(
    pre_burn_ndvi_df["date"],
    pre_burn_ndvi_df["NDVI"],
    label="Pre-fire NDVI",
    color="#FC1FBA",
)
ax1.plot(
    post_burn_ndvi_df["date"],
    post_burn_ndvi_df["NDVI"],
    label="Post-fire NDVI",
    color="#FDAD81",
)
ax1.set_ylabel("NDVI")

ax2 = ax1.twinx()  # instantiate a second Axes that shares the same x-axis
ax2.plot(
    palsar_df["date"], palsar_df["HH_dB"], linewidth=2, label="HH dB", color="#048BA8"
)
ax2.set_ylabel("dB")
ax2.axvline(x=fire_date, color="red", linestyle="--", linewidth=3, label="Fire")

plt.title("Vegetation Recovery Following Fire")
plt.xlabel("Date")

handles_ax1, labels_ax1 = ax1.get_legend_handles_labels()
handles_ax2, labels_ax2 = ax2.get_legend_handles_labels()
handles = handles_ax1 + handles_ax2
labels = labels_ax1 + labels_ax2
plt.legend(handles, labels, loc="lower right")

plt.grid(True)
plt.show()

In [ ]:
# Split the NDVI into two sets:
# 1) A period near when the fire occurred (i.e., rapid recovery)
# 2) A period further away from when the fire occurred (i.e., cover might have recoved)
pre_burn_ndvi = pre_burn_ndvi_df["NDVI"].values
post_p1_burn_ndvi = post_burn_ndvi_df[
    (post_burn_ndvi_df["date"] > "2015-08-20")
    & (post_burn_ndvi_df["date"] <= "2018-01-01")
]["NDVI"].values
post_p2_burn_ndvi = post_burn_ndvi_df[(post_burn_ndvi_df["date"] > "2019-01-01")][
    "NDVI"
].values

In [ ]:
# plotting second histogram
plt.hist(post_p1_burn_ndvi, alpha=0.5, label="Period 1")
# plotting first histogram
plt.hist(pre_burn_ndvi, alpha=0.5, label="Period 2")

# Add legend to the plot
plt.legend()
# Showing the plot using plt.show()
plt.show()

In [ ]:
# Use a Q-Q plot to test the data for normality
# The distribution can be considered to be normally distributed
# is the data points (blue points) largely align with the red line.

scipy.stats.probplot(post_p1_burn_ndvi, dist="norm", plot=plt)
plt.show()

# This would be considered to be normally distributed.

In [ ]:
# Use a Q-Q plot to test the data for normality
# The distribution can be considered to be normally distributed
# is the data points (blue points) largely align with the red line.

scipy.stats.probplot(pre_burn_ndvi, dist="norm", plot=plt)
plt.show()

# This would be considered to be normally distributed.

In [ ]:
# As we know the data are normally distributed we can undertake a t-test
# to assess whether the NDVI pre and post fire are statistically different.
scipy.stats.ttest_ind(pre_burn_ndvi, post_p1_burn_ndvi)


*   The t-statistic of 11.43 is a measure of the difference between the two sample means in terms of standard error. A high absolute value of t indicates a large difference relative to the variability in the data. Therefore, this indicates there is significant difference between the NDVI values before and after the fire.

*   The p-value is extremely small, effectively 0. This indicates that the observed difference between the two sample means is statistically significant. Therefore, you reject the null hypothesis that the NDVI values before and after the fire are identical.

*   Degrees of freedom reflect the sample size and are used to determine the critical value of t for the given test. Here, df=237 suggests a relatively large dataset.

The results suggest strong evidence to reject the null hypothesis that the two samples have identical average (expected) values. There is a statistically significant difference in NDVI values before and after the forest fire.

This suggests that:
1.   The forest fire likely caused a measurable reduction in vegetation health and/or density, as reflected in NDVI values.
2.   The post-fire NDVI values are significantly lower than pre-fire NDVI values, aligning with the expectation that a fire damages vegetation.


If you are unsure how to intepret the results from the t-test then ChatGPT can help, try asking the following:

> "Interpret the output from the t-test TtestResult(statistic=11.431690892741347, pvalue=2.1601506581083213e-24, df=237.0) which compared ndvi values from Landsat imagery before and after a forest fire"


In addition to the t-test it can be useful to calculate the Cohen's d. Cohen's d is a standardized measure of the effect size, which quantifies the magnitude of the difference between two groups in terms of standard deviations. This can helps answer the question:

> "How large is the impact of the forest fire on the NDVI?"

For interpreting the output from the Cohen's d metric the following thresholds are considered standard interpretations:

*   d < 0.2 -  Negligible effect
*   0.2 < d < 0.5 - Small effect
*   0.5 < d < 0.8 - Medium effect
*   d > 0.8 - Large effect


In [ ]:
# The next step is to calculate the Cohen's d:
t_rslt = scipy.stats.ttest_ind(pre_burn_ndvi, post_p1_burn_ndvi)
t_statistic = numpy.abs(t_rslt.statistic)

n1 = pre_burn_ndvi.shape[0]
n2 = post_p2_burn_ndvi.shape[0]

cohens_d = t_statistic / numpy.sqrt(n1 + n2)
print("Cohen's d:", cohens_d)

A Cohen's d of 0.585 would suggest a reasonable ('medium') amount of degradation of vegetation health and density post-fire up until the end of 2017.

In [ ]:
# The next step is to repeat that test but for period 2 (i.e., further away from the fire)
scipy.stats.ttest_ind(pre_burn_ndvi, post_p2_burn_ndvi)


*   A negative t-statistic suggests that the mean NDVI values after the forest fire are likely lower than before the fire. The magnitude of t=2.20 reflects the size of the difference between the group means relative to the variability in the data.
*   The p-value of 0.0284 is less than the typical significance level (α=0.05). This means there is sufficient evidence to reject the null hypothesis that NDVI values before and after the forest fire are identical. The results suggest that the forest fire had a statistically significant impact on NDVI values.


The t-test indicates a statistically significant difference in NDVI values before and after the forest fire (p=0.0284), with lower NDVI values after the fire. This implies that the forest fire caused a measurable reduction in vegetation health or density, as captured by Landsat NDVI values.


Now let's look at the Cohen's d:

In [ ]:
t_rslt = scipy.stats.ttest_ind(pre_burn_ndvi, post_p2_burn_ndvi)
t_statistic = numpy.abs(t_rslt.statistic)

n1 = pre_burn_ndvi.shape[0]
n2 = post_p2_burn_ndvi.shape[0]

cohens_d = t_statistic / numpy.sqrt(n1 + n2)
print("Cohen's d:", cohens_d)

A Cohen's d of 0.115 would suggest a negligible amount of degradation of vegetation health and density post-fire from the beginning of 2018 to the end of 2024.

Therefore, while the t-test indicates that there is a change in the NDVI the Cohen's d indicates that the significance of the change has reduced compared to the period 2015-2018 as we would have expected - i.e., the vegetation is recovering.

In [ ]:
# Split the PALSAR data into pre and post fire periods
pre_burn_palsar = palsar_df[(palsar_df["date"] < "2015-08-20")]["HV_dB"].values
post_burn_palsar = palsar_df[(palsar_df["date"] > "2015-08-20")]["HV_dB"].values

In [ ]:
# Plot the histogram of the PALSAR backscatter pre and post
# the fire.
plt.hist(pre_burn_palsar, alpha=0.5, label="Pre-fire")
plt.hist(post_burn_palsar, alpha=0.5, label="Post-fire")

plt.legend()
# Showing the plot using plt.show()
plt.show()

In [ ]:
# Use a Q-Q plot to test the data for normality
# The distribution can be considered to be normally distributed
# is the data points (blue points) largely align with the red line.

scipy.stats.probplot(pre_burn_palsar, dist="norm", plot=plt)
plt.show()

In [ ]:
# Use a Q-Q plot to test the data for normality
# The distribution can be considered to be normally distributed
# is the data points (blue points) largely align with the red line.

scipy.stats.probplot(post_burn_palsar, dist="norm", plot=plt)
plt.show()

In [ ]:
# Let's calculate the t-test:
scipy.stats.ttest_ind(pre_burn_palsar, post_burn_palsar)

The t-test indicates a statistically significant difference in L-band HV backscatter values before and after the forest fire (p=0.00187). This implies that the forest fire caused a measurable reduction in L-band HV backscatter values, likely due to the destruction of vegetation and loss of biomass.

If you are unsure how to intepret the results from the t-test then ChatGPT can help, try asking the following:

> "Interpret the output from the t-test TtestResult(statistic=4.063658173742494, pvalue=0.0018715134169640304, df=11.0) which compared L-band HV backscatter values from ALOS PALSAR imagery before and after a forest fire"

The Cohen's d works best with larger sample sizes (i.e., over 50 samples) but let's have a look at the value for the L-band backscatter:


In [ ]:
t_rslt = scipy.stats.ttest_ind(pre_burn_palsar, post_burn_palsar)
t_statistic = numpy.abs(t_rslt.statistic)

n1 = pre_burn_ndvi.shape[0]
n2 = post_p2_burn_ndvi.shape[0]

cohens_d = t_statistic / numpy.sqrt(n1 + n2)
print("Cohen's d:", cohens_d)

A Cohen's d of 0.208 would suggest a small amount of degradation of vegetation health and density post-fire. Note that unlike with the NDVI this is all the values from 2015 to 2024 and therefore while the change directly after the fire is a more extreme change this is reduced by the recovery later within the observed period.